# Build the table for Visualization 

Transform format and combine OAG, CFPB and Tweets together. 

In [638]:
import pandas as pd
import geopandas as gpd
import pickle

## OAG

import oag data with firm name

In [388]:
oag_data = pd.read_pickle('max_OAG_firm_list_v2.pkl')

In [389]:
# oag_data.head(100)

In [390]:
oag_data = oag_data[oag_data["COMPLAINANT_STATE"]=='NY']
oag_data = oag_data.drop_duplicates(subset=['MATTER_ID'])
len(oag_data)

8188

In [391]:
# oag_data.columns

In [392]:
oag_df = oag_data.loc[:,['MATTER_ID','cusp_firm_name','COMPLAINT_DATE','COMPLAINANT_ZIP','COMPLAINT_DESCRIPTION']]

In [393]:
oag_df['MATTER_ID'] = oag_df['MATTER_ID'].apply(lambda x: int(''.join(x.split(','))))

In [394]:
oag_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8188 entries, 0 to 12342
Data columns (total 5 columns):
MATTER_ID                8188 non-null int64
cusp_firm_name           8188 non-null object
COMPLAINT_DATE           8188 non-null object
COMPLAINANT_ZIP          8188 non-null int64
COMPLAINT_DESCRIPTION    8186 non-null object
dtypes: int64(2), object(3)
memory usage: 383.8+ KB


import oag data with category label

In [395]:
oag_label = pd.read_pickle('../data/OAG_corpus_with_CUSP_code.pkl')

In [396]:
oag_label = oag_label.loc[:,['MATTER_ID','cusp_code','CUSP_NAAG']]

In [397]:
oag_label = oag_label.drop_duplicates(subset=['MATTER_ID'])

In [398]:
cusp_code_index = oag_label.iloc[:,1:].drop_duplicates(subset=['cusp_code'])

In [399]:
cusp_code_index

,cusp_code,CUSP_NAAG
0,2,scams
1,4,financial
2,8,travel
4,5,good_n_services
5,1,None
7,7,utilities
11,0,Residential
14,3,misc
143,6,government


In [400]:
print len(oag_df)
print len(oag_label)

8188
12416


In [407]:
oag_with_label = oag_df.merge(oag_label, on='MATTER_ID', how = 'inner')

In [408]:
len(oag_with_label)

8166

In [409]:
oag_with_label.dropna(subset=['COMPLAINT_DESCRIPTION'], inplace=True)

In [410]:
oag_with_label = oag_with_label.drop(['MATTER_ID','CUSP_NAAG'], axis=1)

In [412]:
oag_with_label = oag_with_label[oag_with_label['cusp_code'] != 1]

In [414]:
oag_with_label['cusp_code'] = oag_with_label['cusp_code'].replace(0,1)

In [415]:
oag_with_label['cusp_code'].unique()

array([2, 3, 5, 4, 8, 7, 1, 6])

import zip to county FIPS inference

In [426]:
zip_code_table = pd.read_csv('../data/New_York_State_ZIP_Codes-County_FIPS_Cross-Reference.csv')

In [427]:
zip_code_table = zip_code_table.loc[:,["County Name","County FIPS","ZIP Code",'zip3']]

In [428]:
zip_code_table = zip_code_table.dropna(subset=['ZIP Code'])
zip_code_table = zip_code_table.drop_duplicates(subset=['ZIP Code'])

In [429]:
zip_code_table.head()

,County Name,County FIPS,ZIP Code,zip3
0,Albany,36001,12046,120
1,Albany,36001,12083,120
2,Albany,36001,12085,120
3,Albany,36001,12201,122
4,Albany,36001,12203,122


In [432]:
oag_with_county = oag_with_label.merge(zip_code_table, left_on='COMPLAINANT_ZIP', right_on='ZIP Code', how='inner')

In [433]:
len(oag_with_county)

7030

In [434]:
oag_with_county = oag_with_county.loc[:,['cusp_firm_name','cusp_code','COMPLAINT_DATE','County FIPS','COMPLAINT_DESCRIPTION']]

In [437]:
oag_final = oag_with_county

In [609]:
oag_final.columns = ['Company','Category','Date','County_FIPS','Description']

In [624]:
oag_final.loc[:,'Data Source'] = 'OAG Online'

In [672]:
oag_final.Date = oag_final.Date.apply(lambda x: x.split(' ')[0])

In [689]:
for i in range(len(oag_final)):
    original = oag_final.iloc[i,2].split('/')
    if len(original[0]) == 1:
        original[0] = '0' + original[0]
    if len(original[1]) == 1:
        original[1] = '0' + original[1]
    original[2] = '20' + original[2]
    new = '/'.join(original)
    oag_final.iloc[i,2] = new

In [700]:
oag_final.head()

,Company,Category,Date,County_FIPS,Description,Data Source
0,ticketmaster,2,06/07/2016,36061,I attempted to purchase two tickets to see ---...,OAG Online
1,apple company,5,06/07/2016,36061,I had to contact them including - personal tim...,OAG Online
2,planet fitness,5,09/09/2015,36061,Note User indicated supporting documents will ...,OAG Online
3,jason starr,1,05/17/2016,36061,Was hired to paint and do basic construction w...,OAG Online
4,american airlines,8,04/04/2016,36061,Note User indicated supporting documents will ...,OAG Online


In [702]:
oag_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7030 entries, 0 to 7029
Data columns (total 6 columns):
Company        7030 non-null object
Category       7030 non-null int64
Date           7030 non-null object
County_FIPS    7030 non-null int64
Description    7030 non-null object
Data Source    7030 non-null object
dtypes: int64(2), object(4)
memory usage: 384.5+ KB


In [698]:
company_list_oag = list(oag_final.Company.unique())

In [699]:
len(company_list_oag)

5359

## CFPB

In [489]:
cfpb_data = pd.read_csv('../data/maxs_CFPBv1.csv',index_col=0)

In [490]:
len(cfpb_data)

249875

In [491]:
# cfpb_data.columns

In [492]:
cfpb_data.loc[:,'categoary'] = 4

In [502]:
cfpb_ = cfpb_data.loc[:, ['cusp_firm_name','categoary','Date received','ZIP code','Consumer complaint narrative']]

In [503]:
cfpb_.dropna(subset=['Consumer complaint narrative'], inplace=True)
cfpb_.reset_index(drop=True, inplace=True)

In [504]:
cfpb_.loc[:,'zip3'] = cfpb_.loc[:,'ZIP code'].apply(lambda x: int(x[:3]) if x[2] != '-' else False)

In [505]:
cfpb_.dropna(subset=["ZIP code"], inplace=True)

In [506]:
cfpb_.drop(["ZIP code"], axis=1, inplace=True)

In [604]:
# cfpb_.head()

In [508]:
len(cfpb_)

25072

In [509]:
zip_code_table_cfpb = zip_code_table.drop_duplicates(subset=['zip3'])

In [511]:
cfpb_final = cfpb_.merge(zip_code_table_cfpb, on='zip3', how='inner')

In [512]:
len(cfpb_final)

24975

In [515]:
cfpb_final = cfpb_final.loc[:,['cusp_firm_name','categoary','Date received','County FIPS','Consumer complaint narrative']]

In [611]:
cfpb_final.columns = ['Company','Category','Date','County_FIPS','Description']

In [626]:
cfpb_final.loc[:,'Data Source'] = 'CFPB'

In [627]:
cfpb_final.head()

,Company,Category,Date,County_FIPS,Description,Data Source
0,santander bank us,4,04/06/2015,36085,I was contemplating cashing in some US Savings...,CFPB
1,santander bank us,4,05/29/2015,36085,I inadvertently used a Santander account to pa...,CFPB
2,chase,4,04/29/2016,36085,Follow up to complaint # XXXX - Dated XXXX/XXX...,CFPB
3,chase,4,04/29/2016,36085,Follow up to complaint # XXXX - Dated XXXX/XXX...,CFPB
4,chase,4,03/31/2016,36085,I have a major issue with J P Morgan Chase. I ...,CFPB


In [703]:
cfpb_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24975 entries, 0 to 24974
Data columns (total 6 columns):
Company        24975 non-null object
Category       24975 non-null int64
Date           24975 non-null object
County_FIPS    24975 non-null int64
Description    24975 non-null object
Data Source    24975 non-null object
dtypes: int64(2), object(4)
memory usage: 1.3+ MB


In [522]:
company_list_cfpb = list(cfpb_final.cusp_firm_name.unique())

In [659]:
cfpb_final.to_csv('cfpb_date_test.csv')

### firmlist check

In [551]:
print len(company_list_cfpb)
print len(company_list_oag)

445
5359


In [552]:
test = [i for i in company_list_cfpb if i in company_list_oag]

In [553]:
len(test)

48

In [554]:
# sorted(company_list_cfpb)

In [555]:
# sorted(company_list_oag)

In [556]:
firm_df = pd.read_pickle('firmlist.pkl')

In [594]:
firm_df.head()

,firm,cusp_firm_name,length,num_of_words,splits,search
0,ticketmaster,ticketmaster,12,1,[ticketmaster],ticketmaster
1,argo real estate llc,argo real estate,14,4,"[argo, real, estate, llc]",argorealestate
2,acura of westchester,acura of westchester,18,3,"[acura, of, westchester]",acuraofwestchester
3,galaxy moving systems,galaxy moving systems,19,3,"[galaxy, moving, systems]",galaxymovingsystems
4,apple company,apple company,12,2,"[apple, company]",applecompany


In [557]:
firmlist = list(firm_df.cusp_firm_name.unique())

In [558]:
len(firmlist)

12830

In [561]:
check_oag = [i for i in company_list_oag if i not in firmlist]
len(check_oag)

0

In [562]:
check_cfpb = [i for i in company_list_cfpb if i not in firmlist]
len(check_cfpb)

0

In [582]:
whole_firm_list = list(set(company_list_cfpb+company_list_oag))

In [583]:
len(whole_firm_list)

5756

In [590]:
# sorted(whole_firm_list)

In [598]:
mask = [j for j,i in enumerate(list(firm_df['cusp_firm_name'])) if i in whole_firm_list]

In [601]:
firm_df_update = firm_df.iloc[mask,:]

In [602]:
len(firm_df_update)

5756

In [606]:
firm_df_update.head()

,firm,cusp_firm_name,length,num_of_words,splits,search
0,ticketmaster,ticketmaster,12,1,[ticketmaster],ticketmaster
1,argo real estate llc,argo real estate,14,4,"[argo, real, estate, llc]",argorealestate
2,acura of westchester,acura of westchester,18,3,"[acura, of, westchester]",acuraofwestchester
4,apple company,apple company,12,2,"[apple, company]",applecompany
7,time warner cable,time warner,10,2,"[time, warner]",timewarner


In [639]:
pickle.dump(firm_df_update, open('firmlist_update.pkl', 'wb')) 

## Tweet

In [643]:
tweet_df = pd.read_csv('tweets_result.csv', index_col=0)

In [648]:
tweet_df.head()

,tweet,date,GEOID,category,firm
0,@Newsweek instead of firing shots @realDonaldT...,Fri Apr 29 01:28:10 +0000 2016,36081,6,unknown
1,Totally feel awful today. So let's get to pump...,Fri Apr 29 15:18:08 +0000 2016,36081,5,unknown
2,Marketing Campaign Manager - RMS Computer Corp...,Fri Apr 29 19:20:14 +0000 2016,36081,2,unknown
3,If you have had a joint replacement and taken ...,Wed May 11 15:41:47 +0000 2016,36081,5,unknown
4,Time for Zumba session number two! #gymhopping...,Wed May 11 23:07:38 +0000 2016,36081,5,24hourfitness


In [650]:
tweet_final = tweet_df.loc[:,['firm','category','date','GEOID','tweet']]

In [652]:
tweet_final.loc[:, 'datasource'] = 'Twitter'

In [653]:
tweet_final.columns = cfpb_final.columns

In [712]:
firm_df_ = firm_df_update.drop_duplicates(subset=['search'])

In [714]:
tweet_final_ = tweet_final.merge(firm_df_, left_on='Company', right_on='search', how='left')

In [717]:
tweet_final_ = tweet_final_.loc[:,['cusp_firm_name','Category','Date','County_FIPS','Description','Data Source']]

In [721]:
tweet_final_ = tweet_final_.rename(columns={'cusp_firm_name': 'Company'})

In [744]:
Month_list = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [756]:
for i in range(len(tweet_final_)):
    original = tweet_final_.iloc[i,2].split(' ')
    month = str(Month_list.index(original[1])+1)
    if len(month) == 1:
        month = '0' + month
    day = original[2]
    year = original[-1]
    new = '/'.join([month,day,year])
    tweet_final_.iloc[i,2] = new

In [767]:
tweet_final_.head()

,Company,Category,Date,County_FIPS,Description,Data Source
0,unknown,6,04/29/2016,36081,@Newsweek instead of firing shots @realDonaldT...,Twitter
1,unknown,5,04/29/2016,36081,Totally feel awful today. So let's get to pump...,Twitter
2,unknown,2,04/29/2016,36081,Marketing Campaign Manager - RMS Computer Corp...,Twitter
3,unknown,5,05/11/2016,36081,If you have had a joint replacement and taken ...,Twitter
4,24 hour fitness,5,05/11/2016,36081,Time for Zumba session number two! #gymhopping...,Twitter


In [768]:
tweet_final_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 0 to 243
Data columns (total 6 columns):
Company        244 non-null object
Category       244 non-null int64
Date           244 non-null object
County_FIPS    244 non-null int64
Description    244 non-null object
Data Source    244 non-null object
dtypes: int64(2), object(4)
memory usage: 13.3+ KB


# Combine

In [772]:
oag_cfpb_tweet_list = [oag_final, cfpb_final, tweet_final_]
oag_cfpb_tweet = pd.concat(oag_cfpb_tweet_list)

In [848]:
for i in oag_cfpb_tweet_list:
    print len(i)
print len(oag_cfpb_tweet)

7030
24975
244
32249


In [774]:
index_list = ['Residential', 'Scams', 'Miscellaneous', 'Financial', 'Good & Services', 'Government', 'Utilities', 'Travel']

In [776]:
oag_cfpb_tweet['Category'] = oag_cfpb_tweet['Category'].apply(lambda x: index_list[x-1])

In [786]:
oag_cfpb_tweet['Company'] = oag_cfpb_tweet['Company'].apply(lambda x: x.title())

In [795]:
county_infer = zip_code_table.loc[:,['County Name','County FIPS']].drop_duplicates(subset=['County FIPS'])
# county_infer

In [797]:
oag_cfpb_tweet = oag_cfpb_tweet.merge(county_infer, left_on='County_FIPS', right_on='County FIPS' ,how='left')

In [799]:
oag_cfpb_tweet = oag_cfpb_tweet.drop(['County FIPS'], axis=1)

In [812]:
oag_cfpb_tweet.head()

,Company,Category,Date,County_FIPS,Description,Data Source,County Name
0,Ticketmaster,Scams,06/07/2016,36061,I attempted to purchase two tickets to see ---...,OAG Online,New York
1,Apple Company,Good & Services,06/07/2016,36061,I had to contact them including - personal tim...,OAG Online,New York
2,Planet Fitness,Good & Services,09/09/2015,36061,Note User indicated supporting documents will ...,OAG Online,New York
3,Jason Starr,Residential,05/17/2016,36061,Was hired to paint and do basic construction w...,OAG Online,New York
4,American Airlines,Travel,04/04/2016,36061,Note User indicated supporting documents will ...,OAG Online,New York


In [831]:
county_population = pd.read_csv('../data/county population.csv')

In [832]:
county_population['County'] = county_population['County'].apply(lambda x: ' '.join(x.split()[:-1]))
county_population.columns = ['County Name','County Population']

In [839]:
# county_population

In [836]:
oag_cfpb_tweet = oag_cfpb_tweet.merge(county_population, on='County Name', how='left')

In [845]:
oag_cfpb_tweet.head()

,Company,Category,Date,County_FIPS,Description,Data Source,County Name,County Population
0,Ticketmaster,Scams,06/07/2016,36061,I attempted to purchase two tickets to see ---...,OAG Online,New York,1644518.0
1,Apple Company,Good & Services,06/07/2016,36061,I had to contact them including - personal tim...,OAG Online,New York,1644518.0
2,Planet Fitness,Good & Services,09/09/2015,36061,Note User indicated supporting documents will ...,OAG Online,New York,1644518.0
3,Jason Starr,Residential,05/17/2016,36061,Was hired to paint and do basic construction w...,OAG Online,New York,1644518.0
4,American Airlines,Travel,04/04/2016,36061,Note User indicated supporting documents will ...,OAG Online,New York,1644518.0


In [846]:
oag_cfpb_tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32249 entries, 0 to 32248
Data columns (total 8 columns):
Company              32249 non-null object
Category             32249 non-null object
Date                 32249 non-null object
County_FIPS          32249 non-null int64
Description          32249 non-null object
Data Source          32249 non-null object
County Name          32163 non-null object
County Population    32163 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 2.2+ MB


In [847]:
oag_cfpb_tweet.to_csv('oag_cfpb_tweet.csv')